In [2]:
import numpy as np
import time, os, sys
from urllib.parse import urlparse
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

from scipy.ndimage.measurements import center_of_mass as som

mpl.rcParams['figure.dpi'] = 300
from cellpose import models, utils, io

In [3]:


#img = the img u just segmented
#frame = whats the frame #
#omnibus = what list do u want to append too...writign to file probably better 

track_arr = []

def get_centroids(inp_image,frame,omnibus):
    centroid_list = []
    props = measure.regionprops(inp_image)
    for q in range(1,np.unique(inp_image).shape[0] -1):
        #print(q)
        centroid_coords =list(props[q].centroid)
        centroid_coords.insert(0,frame)
        omnibus.append(centroid_coords)


def frame_to_mask(frame):
    # DEFINE CELLPOSE MODEL
    # model_type='cyto' or model_type='nuclei'
    model = models.Cellpose(gpu=False, model_type='cyto')

    chan = [0,0]
    masks, flows, styles, diams = model.eval(frame, channels=chan, diameter = 65, flow_threshold= 0.8,resample = True)
    return masks

def preprocess_frame(chann_ar,frame):
    nuc = normalize99(chann_ar[frame,1,:,:])
    other = normalize99((chann_ar[frame,0,:,:]) + (chann_ar[frame + 1,0,:,:]  + (chann_ar[frame + 2,0,:,:])))
    bright = normalize99(chann_ar[frame,2,:,:])
    new_img = normalize99(nuc - other)
    
    return new_img
def normalize99(img):
    """ normalize image so 0.0 is 1st percentile and 1.0 is 99th percentile """
    X = img.copy()
    X = (X - np.percentile(X, 1)) / (np.percentile(X, 99) - np.percentile(X, 1))
    return X


In [5]:
vascu_frames = "/home/lpe/Desktop/vasc_tsrs/200123_mEos_cells_4.tif"###### Our times series
time_series = io.imread(vascu_frames)

for i in range(time_series.shape[0] -2):
    processed = preprocess_frame(time_series,i)
    frame_mask = frame_to_mask(processed)
    get_centroids(frame_mask,i,track_arr)

>>>> using CPU
Running test snippet to check if MKL-DNN working
see https://pytorch.org/docs/stable/backends.html?highlight=mkl
** MKL version working - CPU version is sped up. **
processing 1 image(s)
time spent: running network 6.86s; flow+mask computation 9.84
estimated masks for 1 image(s) in 16.77 sec
>>>> TOTAL TIME 16.77 sec


NameError: name 'measure' is not defined

In [ ]:
point2track = np.asarray(track_arr)
np.savetxt("/home/lpe/Desktop/cellpose/epithelial_points2track.txt",point2track)